In [1]:
import torch
from typing import Tuple
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
# hyperparameters

SNR = 20
lstsq_method = 'non-linear' # 'linear' or 'non-linear'

In [3]:
# load b-values from data

b_values_path = 'C:/Users/panag/Desktop/Test/mgh_1001/diff/preproc/bvals.txt'
b_values = np.genfromtxt(b_values_path)
b_values[(b_values > 9_900) & (b_values < 10_100)] = 10_000
b_values /= 1_000.0

b_values.shape

(552,)

In [4]:
# load gradients from data

direction_vectors_path = 'C:/Users/panag/Desktop/Test/mgh_1001/diff/preproc/bvecs_moco_norm.txt'
direction_vectors = np.genfromtxt(direction_vectors_path)

direction_vectors.shape

(552, 3)

In [5]:
# load lstsq_results

lstsq_results: dict[Tuple[float, float, float], np.ndarray]

if lstsq_method == 'linear':
    with open('lstsq_results_linear.pkl', 'rb') as f:
        lstsq_results = pickle.load(f)

elif lstsq_method == 'non-linear':
    with open('non_linear_lstsq_results.pkl', 'rb') as f:
        lstsq_results = pickle.load(f)

else:
    raise ValueError(f'Invalid value for lstsq_method: {lstsq_method}')

In [7]:
# calculate d_tensors and noisy_signals

custom_b_values = torch.from_numpy(b_values[(b_values == 0.0) | (b_values == 1.0)]).float()
custom_gradients = torch.from_numpy(direction_vectors[(b_values == 0.0) | (b_values == 1.0)]).float()

d_tensors = []
signals = []
noisy_signals = []

for d_array in tqdm(lstsq_results.values()):

    d_tensor = torch.tensor(d_array).float()
    
    signal = torch.exp(- custom_b_values * torch.einsum('bi, ij, bj -> b', custom_gradients, d_tensor, custom_gradients))
    
    noise = torch.normal(mean=torch.zeros_like(signal), std=torch.ones_like(signal) / SNR)

    noisy_signal = signal + noise

    noisy_signal /= torch.mean(noisy_signal[custom_b_values == 0.0])

    d_tensors.append(d_tensor)
    signals.append(signal)
    noisy_signals.append(noisy_signal)

d_tensors = torch.stack(d_tensors) # shape (num_d_tensors, 3, 3)
signals = torch.stack(signals) # shape (num_d_tensors, len(custom_b_values))
noisy_signals = torch.stack(noisy_signals) # shape (num_d_tensors, len(custom_b_values))

100%|██████████| 338645/338645 [01:00<00:00, 5635.23it/s]


In [15]:
print(f'{d_tensors.shape=}')
print(f'{signals.shape=}, \t {signals.min()=}, \t\t {signals.max()=}')
print(f'{noisy_signals.shape=}, \t {noisy_signals.min()=}, \t {noisy_signals.max()=}')

d_tensors.shape=torch.Size([338645, 3, 3])
signals.shape=torch.Size([338645, 104]), 	 signals.min()=tensor(0.), 		 signals.max()=tensor(1.)
noisy_signals.shape=torch.Size([338645, 104]), 	 noisy_signals.min()=tensor(-0.2005), 	 noisy_signals.max()=tensor(1.2581)


In [9]:
# set file names

d_tensors_file = f'{lstsq_method} sim d_tensors.pt'
signals_file = f'{lstsq_method} sim signals.pt'
noisy_signals_file = f'{lstsq_method} sim noisy_signals {SNR}.pt'

In [10]:
# save d_tensors and noisy_signals

torch.save(d_tensors, d_tensors_file)
torch.save(signals, signals_file)
torch.save(noisy_signals, noisy_signals_file)